# Slack
You need to fill in the [form](https://docs.google.com/forms/d/1OmT8ODmVBNgl0eOmZT51JMTHUSA_eNrHTcDRnmNDMgQ) to get invitated

Slack url: https://rt-portal.slack.com/

# Pandas Cheet Sheat
https://github.com/pandas-dev/pandas/raw/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

In [2]:
from pandas import Series, DataFrame
import pandas as pd

In [3]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

## Essential functionality

In this section, I’ll walk you through the fundamental mechanics of interacting with
the data contained in a Series or DataFrame. Upcoming chapters will delve more deeply
into data analysis and manipulation topics using pandas. This book is not intended to
serve as exhaustive documentation for the pandas library; I instead focus on the most
important features, leaving the less common (that is, more esoteric) things for you to
explore on your own.

### Reindexing

A critical method on pandas objects is **reindex** , which means to create a new object
with the data *conformed* to a new index. Consider a simple example from above:

In [4]:
obj = Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

Calling **reindex** on this Series rearranges the data according to the new index, intro-
ducing missing values if any index values were not already present:

In [5]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [6]:
obj.reindex(['a', 'b', 'c', 'd', 'e'], fill_value=0)

a   -5.3
b    7.2
c    3.6
d    4.5
e    0.0
dtype: float64

For ordered data like time series, it may be desirable to do some interpolation or filling
of values when reindexing. The **method** option allows us to do this, using a method such
as **ffill** which forward fills the values:

In [7]:
obj3 = Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

**Table 5-4 **lists available method options. At this time, interpolation more sophisticated
than forward- and backfilling would need to be applied after the fact.

**Table 5-4. reindex method (interpolation) options**

Argument | Description
--- | ---
ffill or pad | Fill (or carry) values forward
bfill or backfill | Fill (or carry) values backward

With DataFrame, **reindex** can alter either the (row) index, columns, or both. When
passed just a sequence, the rows are reindexed in the result:

In [ ]:
frame = DataFrame(np.arange(9).reshape((3, 3)), index=['a', 'c', 'd'],
                  columns=['Ohio', 'Texas', 'California'])
frame

In [ ]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

The columns can be reindexed using the columns keyword:

In [ ]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

Both can be reindexed in one shot, though interpolation will only apply row-wise (axis
0):

In [ ]:
#frame.reindex(index=['a', 'b', 'c', 'd'], method='ffill', columns=states)
frame.reindex(index=['a', 'b', 'c', 'd'], columns=states).ffill()

As you’ll see soon, reindexing can be done more succinctly by label-indexing with `loc` :

In [ ]:
frame.loc[['a', 'b', 'c', 'd'], states]

**Table 5-5. reindex function arguments**

Argument | Description
--- | ---
index | New sequence to use as index. Can be Index instance or any other sequence-like Python data structure. An Index will be used exactly as is without any copying
method | Interpolation (fill) method, see Table 5-4 for options.
fill_value | Substitute value to use when introducing missing data by reindexing
limit | When forward- or backfilling, maximum size gap to fill
level | Match simple Index on level of MultiIndex, otherwise select subset of
copy | Do not copy underlying data if new index is equivalent to old index. True by default (i.e. always copy data).

### Dropping entries from an axis

Dropping one or more entries from an axis is easy if you have an index array or list
without those entries. As that can require a bit of munging and set logic, the drop
method will return a new object with the indicated value or values deleted from an axis:

In [ ]:
obj = Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
new_obj = obj.drop('c')
new_obj

In [ ]:
obj.drop(['d', 'c'])

With DataFrame, index values can be deleted from either axis:

In [ ]:
data = DataFrame(np.arange(16).reshape((4, 4)),
                 index=['Ohio', 'Colorado', 'Utah', 'New York'],
                 columns=['one', 'two', 'three', 'four'])

In [ ]:
data.drop(['Colorado', 'Ohio'])

In [ ]:
data.drop('two', axis=1)

In [ ]:
data.drop(['two', 'four'], axis=1)

### Indexing, selection, and filtering

Series indexing ( `obj[...]` ) works analogously to NumPy array indexing, except you can
use the Series’s index values instead of only integers. Here are some examples this:

In [8]:
obj = Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj['b']

1.0

In [9]:
obj[1]

1.0

In [10]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [11]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [12]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [13]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

Slicing with labels behaves differently than normal Python slicing in that the endpoint
is inclusive:

In [14]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

Setting using these methods works just as you would expect:

In [15]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

As you’ve seen above, indexing into a DataFrame is for retrieving one or more columns
either with a single value or sequence:

In [ ]:
data = DataFrame(np.arange(16).reshape((4, 4)),
                 index=['Ohio', 'Colorado', 'Utah', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

In [ ]:
data['two']

In [ ]:
data[['three', 'one']]

Indexing like this has a few special cases. First selecting rows by slicing or a boolean
array:

In [ ]:
data[:2]

In [ ]:
data[data['three'] > 5]

This might seem inconsistent to some readers, but this syntax arose out of practicality
and nothing more. Another use case is in indexing with a boolean DataFrame, such as
one produced by a scalar comparison:

In [ ]:
data < 5

In [ ]:
data[data < 5] = 0

In [ ]:
data

This is intended to make DataFrame syntactically more like an ndarray in this case.
For DataFrame label-indexing on the rows, I introduce the special indexing field `loc` . It
enables you to select a subset of the rows and columns from a DataFrame with NumPy-like notation plus axis labels. As I mentioned earlier, this is also a less verbose way to
do reindexing:

http://pandas.pydata.org/pandas-docs/stable/indexing.html#different-choices-for-indexing

In [ ]:
data.loc['Colorado', ['two', 'three']]

In [ ]:
data.ix[['Colorado', 'Utah'], [3, 0, 1]]

In [ ]:
data.iloc[2]

In [ ]:
data.loc[:'Utah', 'two']

In [ ]:
data.loc[data.three > 5, :'two']

**Table 5-6. Indexing options with DataFrame**

Type | Notes
--- | ---
obj[val]  |Select single column or sequence of columns from the DataFrame. Special case conveniences: boolean array (filter rows), slice (slice rows), or boolean DataFrame (set values based on some criterion).
obj.ix[val] | Selects single row of subset of rows from the DataFrame.
obj.ix[:, val] | Selects single column of subset of columns.
obj.ix[val1, val2] | Select both rows and columns. 
reindex method | Conform one or more axes to new indexes.
xs method | Select single row or column as a Series by label.
icol, irow methods | Select single column or row, respectively, as a Series by integer location.
get_value, set_value methods | Select single value by row and column label.

### Arithmetic and data alignment

One of the most important pandas features is the behavior of arithmetic between objects with different indexes. When adding together objects, if any index pairs are not
the same, the respective index in the result will be the union of the index pairs. Let’s
look at a simple example:

In [16]:
s1 = Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])

In [17]:
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [18]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

Adding these together yields:

In [ ]:
s1 + s2

The internal data alignment introduces NA values in the indices that don’t overlap.
Missing values propagate in arithmetic computations.
In the case of DataFrame, alignment is performed on both the rows and the columns:

In [ ]:
df1 = DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                index=['Ohio', 'Texas', 'Colorado'])
df2 = DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df1

In [ ]:
df2

Adding these together returns a DataFrame whose index and columns are the unions
of the ones in each DataFrame:

In [ ]:
df1 + df2

#### Arithmetic methods with fill values

In arithmetic operations between differently-indexed objects, you might want to fill
with a special value, like 0, when an axis label is found in one object but not the other:

In [ ]:
df1 = DataFrame(np.arange(12.).reshape((3, 4)), columns=list('abcd'))
df2 = DataFrame(np.arange(20.).reshape((4, 5)), columns=list('abcde'))
df1

In [ ]:
df2

Adding these together results in NA values in the locations that don’t overlap:

In [ ]:
df1 + df2

Using the add method on df1 , I pass df2 and an argument to fill_value :

In [ ]:
df1.add(df2, fill_value=0)

Relatedly, when reindexing a Series or DataFrame, you can also specify a different fill
value:

In [ ]:
df1.reindex(columns=df2.columns, fill_value=0)

**Table 5-7. Flexible arithmetic methods**

Method | Description
--- | ---
add | Method for addition (+)
sub | Method for subtraction (-)
div | Method for division (/)
mul | Method for multiplication (*)

#### Operations between DataFrame and Series

As with NumPy arrays, arithmetic between DataFrame and Series is well-defined. First,
as a motivating example, consider the difference between a 2D array and one of its rows:

In [ ]:
arr = np.arange(12.).reshape((3, 4))
arr

In [ ]:
arr[0]

In [ ]:
arr - arr[0]

This is referred to as broadcasting and is explained in more detail in Chapter 12. Op-
erations between a DataFrame and a Series are similar:

In [ ]:
frame = DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.iloc[0]
frame

In [ ]:
series

By default, arithmetic between DataFrame and Series matches the index of the Series
on the DataFrame's columns, broadcasting down the rows:

In [ ]:
frame - series

If an index value is not found in either the DataFrame’s columns or the Series’s index,
the objects will be reindexed to form the union:

In [ ]:
series2 = Series(range(3), index=['b', 'e', 'f'])
frame + series2

If you want to instead broadcast over the columns, matching on the rows, you have to
use one of the arithmetic methods. For example:

In [ ]:
series3 = frame['d']
frame

In [ ]:
series3

In [ ]:
frame.sub(series3, axis=0)

The axis number that you pass is the axis to match on. In this case we mean to match
on the DataFrame’s row index and broadcast across.

### Function application and mapping

NumPy ufuncs (element-wise array methods) work fine with pandas objects:

In [ ]:
frame = DataFrame(np.random.randn(4, 3), columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [ ]:
frame

In [ ]:
np.abs(frame)

Another frequent operation is applying a function on 1D arrays to each column or row.
DataFrame’s **apply** method does exactly this:

In [ ]:
f = lambda x: x.max() - x.min()

In [ ]:
frame.apply(f)

In [ ]:
frame.apply(f, axis=1)

Many of the most common array statistics (like **sum** and **mean** ) are DataFrame methods,
so using **apply** is not necessary.
The function passed to **apply** need not return a scalar value, it can also return a Series
with multiple values:

In [ ]:
def f(x):
    return Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

Element-wise Python functions can be used, too. Suppose you wanted to compute a
formatted string from each floating point value in **frame** . You can do this with **applymap** :

In [ ]:
format = lambda x: '%.2f' % x
frame.applymap(format)

In [ ]:
frame['e'].map(format)

### Sorting and ranking

Sorting a data set by some criterion is another important built-in operation. To sort
lexicographically by row or column index, use the sort_index method, which returns
a new, sorted object:

In [ ]:
obj = Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index()

With a DataFrame, you can sort by index on either axis:

In [ ]:
frame = DataFrame(np.arange(8).reshape((2, 4)), index=['three', 'one'],
                  columns=['d', 'a', 'b', 'c'])
frame.sort_index()

In [ ]:
frame.sort_index(axis=1)

The data is sorted in ascending order by default, but can be sorted in descending order,
too:

In [ ]:
frame.sort_index(axis=1, ascending=False)

To sort a Series by its values, use its order method:

In [ ]:
obj = Series([4, 7, -3, 2])
obj.order()

Any missing values are sorted to the end of the Series by default:

In [ ]:
obj = Series([4, np.nan, 7, np.nan, -3, 2])
obj.order()

On DataFrame, you may want to sort by the values in one or more columns. To do so,
pass one or more column names to the by option:

In [ ]:
frame = DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

In [ ]:
frame.sort_index(by='b')

To sort by multiple columns, pass a list of names:

In [ ]:
frame.sort_index(by=['a', 'b'])

*Ranking* is closely related to sorting, assigning ranks from one through the number of
valid data points in an array. It is similar to the indirect sort indices produced by
**numpy.argsort** , except that ties are broken according to a rule. The **rank** methods for
Series and DataFrame are the place to look; by default **rank** breaks ties by assigning
each group the mean rank:

In [ ]:
obj = Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

Ranks can also be assigned according to the order they’re observed in the data:

In [ ]:
obj.rank(method='first')

Naturally, you can rank in descending order, too:

In [ ]:
obj.rank(ascending=False, method='max')

See **Table 5-8** for a list of tie-breaking methods available. DataFrame can compute ranks
over the rows or the columns:

In [ ]:
frame = DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                   'c': [-2, 5, 8, -2.5]})
frame

In [ ]:
frame.rank(axis=1)

**Table 5-8. Tie-breaking methods with rank**

Method | Description
--- | ---
'average' | Default: assign the average rank to each entry in the equal group.
'min' | Use the minimum rank for the whole group.
'max' | Use the maximum rank for the whole group.
'first' | Assign ranks in the order the values appear in the data.

### Axis indexes with duplicate values

Up until now all of the examples I’ve showed you have had unique axis labels (index
values). While many pandas functions (like reindex ) require that the labels be unique,
it’s not mandatory. Let’s consider a small Series with duplicate indices:

In [ ]:
obj = Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

The index’s is_unique property can tell you whether its values are unique or not:

In [ ]:
obj.index.is_unique

Data selection is one of the main things that behaves differently with duplicates. Indexing a value with multiple entries returns a Series while single entries return a scalar
value:

In [ ]:
obj['a']

In [ ]:
obj['c']

The same logic extends to indexing rows in a DataFrame:

In [ ]:
df = DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

In [ ]:
df.loc['b']

## Summarizing and computing descriptive statistics

pandas objects are equipped with a set of common mathematical and statistical methods. Most of these fall into the category of *reductions* or *summary statistics*, methods
that extract a single value (like the sum or mean) from a Series or a Series of values from
the rows or columns of a DataFrame. Compared with the equivalent methods of vanilla
NumPy arrays, they are all built from the ground up to exclude missing data. Consider
a small DataFrame:

In [ ]:
df = DataFrame([[1.4, np.nan], [7.1, -4.5],
                [np.nan, np.nan], [0.75, -1.3]],
               index=['a', 'b', 'c', 'd'],
               columns=['one', 'two'])
df

Calling DataFrame’s sum method returns a Series containing column sums:

In [ ]:
df.sum()

Passing axis=1 sums over the rows instead:

In [ ]:
df.sum(axis=1)

NA values are excluded unless the entire slice (row or column in this case) is NA. This
can be disabled using the skipna option:

In [ ]:
df.mean(axis=1, skipna=False)

See **Table 5-9** for a list of common options for each reduction method options.

**Table 5-9. Options for reduction methods**

Method | Description
--- | ---
axis | Axis to reduce over. 0 for DataFrame’s rows and 1 for columns.
skipna | Exclude missing values, True by default.
level | Reduce grouped by level if the axis is hierarchically-indexed (MultiIndex).

Some methods, like idxmin and idxmax , return indirect statistics like the index value
where the minimum or maximum values are attained:

In [ ]:
df.idxmax()

In [ ]:
df.apply(np.argmax)

In [ ]:
df.cumsum()

Another type of method is neither a reduction nor an accumulation. describe is one
such example, producing multiple summary statistics in one shot:

In [ ]:
df.describe()

On non-numeric data, describe produces alternate summary statistics:

In [ ]:
obj = Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

See **Table 5-10** for a full list of summary statistics and related methods.

**Table 5-10. Descriptive and summary statistics**

Method | Description
--- | ---
count | Number of non-NA values
describe | Compute set of summary statistics for Series or each DataFrame column
min, max | Compute minimum and maximum values
argmin, argmax | Compute index locations (integers) at which minimum or maximum value obtained, respectively
idxmin, idxmax | Compute index values at which minimum or maximum value obtained, respectively
quantile | Compute sample quantile ranging from 0 to 1
sum | Sum of values
mean | Mean of values
median | Arithmetic median (50% quantile) of values
mad | Mean absolute deviation from mean value
var | Sample variance of values
std | Sample standard deviation of values
skew | Sample skewness (3rd moment) of values
kurt | Sample kurtosis (4th moment) of values
cumsum | Cumulative sum of values
cummin, cummax | Cumulative minimum or maximum of values, respectively
cumprod | Cumulative product of values
diff | Compute 1st arithmetic difference (useful for time series)
pct_change | Compute percent changes

### Correlation and covariance

Some summary statistics, like correlation and covariance, are computed from pairs of
arguments. Let’s consider some DataFrames of stock prices and volumes obtained from
Yahoo! Finance:

In [24]:
import pandas_datareader.data as pdweb
from pandas_datareader import data as pdr
import fix_yahoo_finance # must pip install first 

In [25]:
all_data = {}
for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']:
    all_data[ticker] = pdr.get_data_yahoo(ticker)

price = DataFrame({tic: data['Adj Close']
                   for tic, data in all_data.iteritems()})
volume = DataFrame({tic: data['Volume']
                    for tic, data in all_data.iteritems()})

https://query1.finance.yahoo.com/v7/finance/download/AAPL?period1=-631162800&period2=1496143234&interval=1d&events=history&crumb=lKCFaJWKLfA
https://query1.finance.yahoo.com/v7/finance/download/IBM?period1=-631162800&period2=1496143235&interval=1d&events=history&crumb=lKCFaJWKLfA
https://query1.finance.yahoo.com/v7/finance/download/MSFT?period1=-631162800&period2=1496143236&interval=1d&events=history&crumb=lKCFaJWKLfA
https://query1.finance.yahoo.com/v7/finance/download/GOOG?period1=-631162800&period2=1496143237&interval=1d&events=history&crumb=lKCFaJWKLfA


I now compute percent changes of the prices:

In [26]:
returns = price.pct_change()
returns.tail()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2017-05-22,0.006076,0.008405,0.004343,0.011228
2017-05-23,-0.001234,0.007390,-0.003996,0.003360
2017-05-24,-0.002991,0.006471,0.003157,0.001310
2017-05-25,0.003456,0.015268,0.004524,0.012360
2017-05-26,-0.001690,0.001991,-0.004634,0.004884


The **corr** method of Series computes the correlation of the overlapping, `non-NA`,
aligned-by-index values in two `Series`. Relatedly, **cov** computes the covariance:

In [28]:
returns.MSFT.corr(returns.IBM)

0.44025548819695415

In [29]:
returns.MSFT.cov(returns.IBM)

0.00016634235817470096

DataFrame’s **corr** and **cov** methods, on the other hand, return a full correlation or
covariance matrix as a DataFrame, respectively:

In [31]:
returns.corr()

,AAPL,GOOG,IBM,MSFT
AAPL,1.000000,0.449767,0.364506,0.393151
GOOG,0.449767,1.000000,0.392089,0.449959
IBM,0.364506,0.392089,1.000000,0.440255
MSFT,0.393151,0.449959,0.440255,1.000000


In [32]:
returns.cov()

,AAPL,GOOG,IBM,MSFT
AAPL,0.000858,0.000187,0.000180,0.000245
GOOG,0.000187,0.000382,0.000102,0.000146
IBM,0.000180,0.000102,0.000251,0.000166
MSFT,0.000245,0.000146,0.000166,0.000475


Using DataFrame’s **corrwith** method, you can compute pairwise correlations between
a DataFrame’s columns or rows with another Series or DataFrame. Passing a Series
returns a Series with the correlation value computed for each column:

In [33]:
returns.corrwith(returns.IBM)

AAPL    0.364506
GOOG    0.392089
IBM     1.000000
MSFT    0.440255
dtype: float64

Passing a DataFrame computes the correlations of matching column names. Here I
compute correlations of percent changes with volume:

In [34]:
returns.corrwith(volume)

AAPL    0.007928
GOOG    0.049389
IBM    -0.004195
MSFT   -0.002817
dtype: float64

Passing `axis=1` does things row-wise instead. In all cases, the data points are aligned by
label before computing the correlation.

### Unique values, value counts, and membership

Another class of related methods extracts information about the values contained in a
one-dimensional Series. To illustrate these, consider this example:

In [35]:
obj = Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

The first function is **unique**, which gives you an array of the unique values in a Series:

In [36]:
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

The unique values are not necessarily returned in sorted order, but could be sorted after
he fact if needed (**uniques.sort()**). Relatedly, **value_counts** computes a Series containing value frequencies:

In [37]:
obj.value_counts()

c    3
a    3
b    2
d    1
dtype: int64

The Series is sorted by value in descending order as a convenience. value_counts is also
available as a top-level pandas method that can be used with any array or sequence:

In [38]:
pd.value_counts(obj.values, sort=False)

a    3
c    3
b    2
d    1
dtype: int64

Lastly, isin is responsible for vectorized set membership and can be very useful in
filtering a data set down to a subset of values in a Series or column in a DataFrame:

In [40]:
mask = obj.isin(['b', 'c'])
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [43]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

**Table 5-11. Unique, value counts, and binning methods**

Method | Description
--- | ---
isin | Compute boolean array indicating whether each Series value is contained in the passed sequence of values.
unique | Compute array of unique values in a Series, returned in the order observed.
value_counts | Return a Series containing unique values as its index and frequencies as its values, ordered count in descending order.

In some cases, you may want to compute a histogram on multiple related columns in
a DataFrame. Here’s an example:

In [44]:
data = DataFrame({'Qu1': [1, 3, 4, 3, 4],
                  'Qu2': [2, 3, 1, 2, 3],
                  'Qu3': [1, 5, 2, 4, 4]})
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


Passing **pandas.value_counts** to this DataFrame’s **apply** function gives:

In [45]:
result = data.apply(pd.value_counts).fillna(0)
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0


## Handling missing data

Missing data is common in most data analysis applications. One of the goals in de-
signing pandas was to make working with missing data as painless as possible. For
example, all of the descriptive statistics on pandas objects exclude missing data as
you’ve seen earlier in the chapter.<br>
pandas uses the floating point value **NaN** (Not a Number) to represent missing data in
both floating as well as in non-floating point arrays. It is just used as a sentinel that can
be easily detected:

In [55]:
string_data = Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [56]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

The built-in Python **None** value is also treated as **NA** in object arrays:

In [58]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

I do not claim that pandas’s NA representation is optimal, but it is simple and reasonably consistent. It’s the best solution, with good all-around performance characteristics
and a simple API, that I could concoct in the absence of a true NA data type or bit
pattern in NumPy’s data types. Ongoing development work in NumPy may change this
in the future.

**Table 5-12. NA handling methods**

Argument | Description
--- | ---
dropna | Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate.
fillna | Fill in missing data with some value or using an interpolation method such as 'ffill' or 'bfill' .
isnull | Return like-type object containing boolean values indicating which values are missing / NA. notnull Negation of isnull .

### Filtering out missing data

You have a number of options for filtering out missing data. While doing it by hand is
always an option, dropna can be very helpful. On a Series, it returns the Series with only
the non-null data and index values

In [49]:
from numpy import nan as NA
data = Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

Naturally, you could have computed this yourself by boolean indexing:

In [50]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, these are a bit more complex. You may want to drop rows
or columns which are all NA or just those containing any NAs. dropna by default drops
any row containing a missing value:

In [51]:
data = DataFrame([[1., 6.5, 3.], [1., NA, NA],
                  [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Passing `how='all'` will only drop rows that are all NA:

In [52]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Dropping columns in the same way is only a matter of passing `axis=1` :

In [53]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [54]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [ ]:
data.dropna(axis=1, how='all')

A related way to filter out DataFrame rows tends to concern time series data. Suppose
you want to keep only rows containing a certain number of observations. You can
indicate this with the **thresh** argument:

In [59]:
df = DataFrame(np.random.randn(7, 3))
df.ix[:4, 1] = NA; df.ix[:2, 2] = NA
df

/home/juk/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  from ipykernel import kernelapp as app


,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,NaN
3,1.246435,NaN,-1.296221
4,0.274992,NaN,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [60]:
df.dropna(thresh=3)

,0,1,2
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### Filling in missing data

Rather than filtering out missing data (and potentially discarding other data along with
it), you may want to fill in the “holes” in any number of ways. For most purposes, the
**fillna** method is the workhorse function to use. Calling **fillna** with a constant replaces
missing values with that value:

In [61]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.000000
3,1.246435,0.000000,-1.296221
4,0.274992,0.000000,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


Calling **fillna** with a dict you can use a different fill value for each column:

In [62]:
df.fillna({1: 0.5, 3: -1})

,0,1,2
0,-0.204708,0.500000,NaN
1,-0.555730,0.500000,NaN
2,0.092908,0.500000,NaN
3,1.246435,0.500000,-1.296221
4,0.274992,0.500000,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


**fillna** returns a new object, but you can modify the existing object in place:

In [63]:
# always returns a reference to the filled object
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.000000
3,1.246435,0.000000,-1.296221
4,0.274992,0.000000,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


The same interpolation methods available for reindexing can be used with **fillna** :

In [ ]:
df = DataFrame(np.random.randn(6, 3))
df.ix[2:, 1] = NA; df.ix[4:, 2] = NA
df

In [ ]:
df.fillna(method='ffill')

In [ ]:
df.fillna(method='ffill', limit=2)

With **fillna** you can do lots of other things with a little creativity. For example, you
might pass the mean or median value of a Series:

In [64]:
data = Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

**Table 5-13. fillna function arguments**

Argument | Description
--- | ---
value | Scalar value or dict-like object to use to fill missing values
method | Interpolation, by default 'ffill' if function called with no other arguments
axis | Axis to fill on, default axis=0
inplace | Modify the calling object without producing a copy
limit | For forward and backward filling, maximum number of consecutive periods to fill

## Hierarchical indexing

Hierarchical indexing is an important feature of pandas enabling you to have multiple
(two or more) index levels on an axis. Somewhat abstractly, it provides a way for you
to work with higher dimensional data in a lower dimensional form. Let’s start with a
simple example; create a Series with a list of lists or arrays as the index:

In [65]:
data = Series(np.random.randn(10),
              index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                     [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])
data

a  1    0.476985
   2    3.248944
   3   -1.021228
b  1   -0.577087
   2    0.124121
   3    0.302614
c  1    0.523772
   2    0.000940
d  2    1.343810
   3   -0.713544
dtype: float64

What you’re seeing is a prettified view of a Series with a **MultiIndex** as its index. The
“gaps” in the index display mean “use the label directly above”:

In [ ]:
data.index

With a hierarchically-indexed object, so-called partial indexing is possible, enabling
you to concisely select subsets of the data:

In [ ]:
data['b']

In [ ]:
data['b':'c']

In [ ]:
data.ix[['b', 'd']]

Selection is even possible in some cases from an “inner” level:

In [ ]:
data[:, 2]

Hierarchical indexing plays a critical role in reshaping data and group-based operations
like forming a pivot table. For example, this data could be rearranged into a DataFrame
using its **unstack** method:

In [ ]:
data.unstack()

The inverse operation of **unstack** is **stack** :

In [ ]:
data.unstack().stack()

**stack** and unstack will be explored in more detail in Chapter 7.<br>
With a DataFrame, either axis can have a hierarchical index:

In [ ]:
frame = DataFrame(np.arange(12).reshape((4, 3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'Colorado'],
                           ['Green', 'Red', 'Green']])
frame

The hierarchical levels can have names (as strings or any Python objects). If so, these
will show up in the console output (don’t confuse the index names with the axis labels!):

In [ ]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

With partial column indexing you can similarly select groups of columns:

In [ ]:
frame['Ohio']

A **MultiIndex** can be created by itself and then reused; the columns in the above Data-
Frame with level names could be created like this:

### Reordering and sorting levels

At times you will need to rearrange the order of the levels on an axis or sort the data
by the values in one specific level. The swaplevel takes two level numbers or names and
returns a new object with the levels interchanged (but the data is otherwise unaltered):

In [ ]:
frame.swaplevel('key1', 'key2')

**sortlevel**, on the other hand, sorts the data (stably) using only the values in a single
level. When swapping levels, it’s not uncommon to also use **sortlevel** so that the result
is lexicographically sorted:

In [ ]:
frame.sortlevel(1)

In [ ]:
frame.swaplevel(0, 1).sortlevel(0)

### Summary statistics by level

Many descriptive and summary statistics on DataFrame and Series have a level option
in which you can specify the level you want to sum by on a particular axis. Consider
the above DataFrame; we can sum by level on either the rows or columns like so:

In [ ]:
frame.sum(level='key2')

In [ ]:
frame.sum(level='color', axis=1)

Under the hood, this utilizes pandas’s **groupby** machinery which will be discussed in
more detail later in the book.

### Using a DataFrame's columns

It’s not unusual to want to use one or more columns from a DataFrame as the row
index; alternatively, you may wish to move the row index into the DataFrame’s col-
umns. Here’s an example DataFrame:

In [ ]:
frame = DataFrame({'a': range(7), 'b': range(7, 0, -1),
                   'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                   'd': [0, 1, 2, 0, 1, 2, 3]})
frame

DataFrame’s set_index function will create a new DataFrame using one or more of its
columns as the index:

In [ ]:
frame2 = frame.set_index(['c', 'd'])
frame2

By default the columns are removed from the DataFrame, though you can leave them in:

In [ ]:
frame.set_index(['c', 'd'], drop=False)

**reset_index**, on the other hand, does the opposite of **set_index**; the hierarchical index
levels are are moved into the columns:

In [ ]:
frame2.reset_index()

## Other pandas topics

Here are some additional topics that may be of use to you in your data travels.

### Integer indexing

In [ ]:
ser = Series(np.arange(3.))
ser.iloc[-1]

In [ ]:
ser

In [ ]:
ser2 = Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

In [ ]:
ser.ix[:1]

In [ ]:
ser3 = Series(range(3), index=[-5, 1, 3])
ser3.iloc[2]

In [ ]:
frame = DataFrame(np.arange(6).reshape((3, 2)), index=[2, 0, 1])
frame.iloc[0]

### Panel data

In [70]:
import pandas_datareader.data as pdweb
from pandas_datareader import data as pdr
import fix_yahoo_finance # must pip install first 

While not a major topic of this book, pandas has a Panel data structure, which you can
think of as a three-dimensional analogue of DataFrame. Much of the development focus
of pandas has been in tabular data manipulations as these are easier to reason about,and hierarchical indexing makes using truly N-dimensional arrays unnecessary in a lot
of cases.<br>
To create a Panel, you can use a dict of DataFrame objects or a three-dimensional
ndarray:

In [91]:
pdata = pd.Panel(dict((stk, pdweb.get_data_yahoo(stk))
                       for stk in ['AAPL', 'GOOG', 'MSFT']))

https://query1.finance.yahoo.com/v7/finance/download/AAPL?period1=-631162800&period2=1496148496&interval=1d&events=history&crumb=8OWRIkA0qnO
https://query1.finance.yahoo.com/v7/finance/download/GOOG?period1=-631162800&period2=1496148498&interval=1d&events=history&crumb=8OWRIkA0qnO
https://query1.finance.yahoo.com/v7/finance/download/MSFT?period1=-631162800&period2=1496148499&interval=1d&events=history&crumb=8OWRIkA0qnO


Each item (the analogue of columns in a DataFrame) in the Panel is a DataFrame:

In [95]:
pdata

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 9192 (major_axis) x 3 (minor_axis)
Items axis: Open to Volume
Major_axis axis: 1980-12-12 00:00:00 to 2017-05-26 00:00:00
Minor_axis axis: AAPL to MSFT

In [93]:
pdata = pdata.swapaxes('items', 'minor')
pdata['Adj Close']

/home/juk/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  if __name__ == '__main__':


,AAPL,GOOG,MSFT
Date,,,
1980-12-12,0.513393,NaN,NaN
1980-12-15,0.486607,NaN,NaN
1980-12-16,0.450893,NaN,NaN
1980-12-17,0.462054,NaN,NaN
1980-12-18,0.475446,NaN,NaN
1980-12-19,0.504464,NaN,NaN
1980-12-22,0.529018,NaN,NaN
1980-12-23,0.551339,NaN,NaN
1980-12-24,0.580357,NaN,NaN


**ix**-based label indexing generalizes to three dimensions, so we can select all data at a
particular date or a range of dates like so:

In [ ]:
pdata.ix[:, '6/1/2012', :]

In [ ]:
pdata.ix['Adj Close', '5/22/2012':, :]

An alternate way to represent panel data, especially for fitting statistical models, is in
“stacked” DataFrame form:

In [ ]:
stacked = pdata.ix[:, '5/30/2012':, :].to_frame()
stacked

DataFrame has a related **to_panel** method, the inverse of **to_frame**:

In [ ]:
stacked.to_panel()